<a href="https://colab.research.google.com/github/Sumit217/CMIPtools/blob/master/training_ner_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install scikit-learn
!pip install nltk
!pip install sklearn_crfsuite
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from itertools import chain
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import pandas as pd
import numpy as np
from joblib import dump, load
from nltk.tokenize import word_tokenize
from itertools import islice
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
# !pwd
df = pd.read_csv('/content/Test_Data_News.csv', engine='python', error_bad_lines=False, skiprows = [25417, 507, 13255, 16833, 18377])
df


,Text,Name
0,KOLKATA: The police have registered a case aga...,NaN
1,GUWAHATI: Incessant rain since Thursday night ...,Ganeshguri
2,AGARTALA: The Tripura government has decided t...,NaN
3,BENGALURU: The BJP on Thursday released its fi...,"Karadi Sanganna,HN Ananth Kumar,Tejaswini,Sure..."
4,"NEW DELHI: The Bharatiya Janata Party (BJP), o...","Kanna Lakshminarayana,Narendra Modi,Amit Shah,..."
5,AGARTALA: A division bench of the Tripura high...,"Arindam Lodh,Arun Bhowmik"
6,BENGALURU: From a boy who was groomed by late ...,"Surya,Kumar?Filling,Ananth Kumar,Narendra Modi..."
7,AGARTALA: BJP general secretary Ram Madhav sai...,"Ram Madhav,Narendra Modi"
8,AGARTALA: The anti-tobacco lobby in Tripura ha...,NaN
9,AGARTALA: The charred body of a 30-year-old wo...,"Matilal Sarkar,Tinku Chowdhury,Matilal"


In [0]:

df = df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
# df = df.drop_duplicates()
df[:3] #4511 rows
len(df)
  


17450

In [0]:
def remove_special_characters(string):
    """
    Tokenization/string cleaning for all reviews
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", str(string))
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r'[^\x00-\x7F]+', "",string)
    string = re.sub(" \d+", "",string) 
    string = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ",string)  
    string = re.sub(r"-","",string)
    return string
  
def remove_stopwords(text):
    stops = set(stopwords.words("english"))
    words = [w for w in text.lower().split() if not w in stops]
    words = " ".join(words)
#     print(words)
    return words


In [0]:
%%time

res_list = []
res_word = []

for index, row in df.iterrows(): #.head()
# for index, row in islice(df.iterrows(), 100):
#     print("rows",row)
    text_split = remove_special_characters(row['Text'])
    text_split = remove_stopwords(text_split)
    text_split = word_tokenize(text_split)
#     text_split = word_tokenize(row['Text'])
#     print(text_split)
    name_split = remove_special_characters(row['Name'])
    name_split = remove_stopwords(name_split)
    name_split = name_split.split(',')

    
    for idx, word in enumerate(name_split):
#         print(word.split())
        
        for x in text_split:
            if len(word.split()) == 1:
                  if word.split()[0] in x:
                       res_word.append((x,  nltk.tag.pos_tag([str(x)])[0][1],'B-PER'))
#                        res_list.append(list(set(res_word)))
            
            if len(word.split()) == 2:
                  if word.split()[0] in x:
                       res_word.append((x,  nltk.tag.pos_tag([str(x)])[0][1],'B-PER'))
#                        res_list.append(list(set(res_word)))

                  if word.split()[1] in x:
                       res_word.append((x, nltk.tag.pos_tag([str(x)])[0][1],'I-PER'))
#                        res_list.append(list(set(res_word)))
                      
            if len(word.split()) == 3:
                  if word.split()[0] in x:
                       res_word.append((x,  nltk.tag.pos_tag([str(x)])[0][1],'B-PER'))
#                        res_list.append(list(set(res_word)))

                  if word.split()[1] in x:
                       res_word.append((x, nltk.tag.pos_tag([str(x)])[0][1],'I-PER'))
#                        res_list.append(list(set(res_word)))

                  if word.split()[2] in x:  
                       res_word.append((x,  nltk.tag.pos_tag([str(x)])[0][1],'B-PER')) #eg C V Raman  
#                        res_list.append(list(set(res_word)))
            else: 
                       res_word.append((x,  nltk.tag.pos_tag([str(x)])[0][1], 'O'))
#                        res_list.append(list(set(res_word)))

    res_list.append(res_word)

# print(res_list)    

# B-PER and I-PER indicate the firs last or middle name of the entity and O represent other words in the news article


CPU times: user 40min 44s, sys: 1min 41s, total: 42min 25s
Wall time: 42min 25s


In [0]:

train_sents,test_sents = train_test_split(res_list,test_size=0.2)


In [0]:
test_sents[:8]

[[('guwahati', 'NN', 'O'),
  ('incessant', 'NN', 'O'),
  ('rain', 'NN', 'O'),
  ('since', 'IN', 'O'),
  ('thursday', 'NN', 'O'),
  ('night', 'NN', 'O'),
  ('flooded', 'VBN', 'O'),
  ('large', 'JJ', 'O'),
  ('parts', 'NNS', 'O'),
  ('city', 'NN', 'O'),
  ('bringing', 'VBG', 'O'),
  ('life', 'NN', 'O'),
  ('near', 'IN', 'O'),
  ('standstill', 'NN', 'O'),
  ('friday', 'NN', 'O'),
  ('morning', 'NN', 'O'),
  ('familiar', 'JJ', 'O'),
  ('sights', 'NNS', 'O'),
  ('people', 'NNS', 'O'),
  ('wading', 'VBG', 'O'),
  ('water', 'NN', 'O'),
  ('completed', 'VBN', 'O'),
  ('picture', 'NN', 'O'),
  ('misery', 'NN', 'O'),
  ('run', 'VB', 'O'),
  ('monsoon', 'NN', 'O'),
  ('prime', 'NN', 'O'),
  ('locations', 'NNS', 'O'),
  ('along', 'IN', 'O'),
  ('guwahati', 'NN', 'O'),
  ('shillong', 'NN', 'O'),
  ('road', 'NN', 'O'),
  (',', ',', 'O'),
  ('zoo', 'NN', 'O'),
  ('road', 'NN', 'O'),
  ('stretches', 'NNS', 'O'),
  ('ambari', 'NN', 'O'),
  ('chandmari', 'NN', 'O'),
  ('inundated', 'VBN', 'O'),
  ('deni

In [0]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]



In [0]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]


In [0]:
print(len(y_test),len(X_test))

In [0]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=150,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

In [0]:
labels = list(crf.classes_)
# labels.remove('O')
labels

In [0]:
print(test_sents[3])
# print(X_test[2])
y_pred = crf.predict(X_test[3])
print(y_pred)
dump(crf, 'news_ner_model.joblib') 


In [0]:
# Prediction code

def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

#Prediction Code

import nltk
from joblib import dump, load
crf = load('news_ner_model (3).joblib') 
X_test1 = "While returning from Srinagar, the aircraft carrying other senior Opposition leaders along with Rahul Gandhi"
from nltk.tokenize import word_tokenize 
pos_list = []
text_split = word_tokenize(X_test1)
for x in text_split:
    pos_list.append((x,  nltk.tag.pos_tag([str(x)])[0][1]))
    
print(pos_list)
X_pred_features = [sent2features(pos_list)]
print(X_pred_features)
y_pred_sent = crf.predict(X_pred_features)


print(y_pred_sent)


[('While', 'IN'), ('returning', 'VBG'), ('from', 'IN'), ('Srinagar', 'NN'), (',', ','), ('the', 'DT'), ('aircraft', 'NN'), ('carrying', 'VBG'), ('other', 'JJ'), ('senior', 'JJ'), ('Opposition', 'NN'), ('leaders', 'NNS'), ('along', 'IN'), ('with', 'IN'), ('Rahul', 'NN'), ('Gandhi', 'NNP')]
[[{'bias': 1.0, 'word.lower()': 'while', 'word[-3:]': 'ile', 'word[-2:]': 'le', 'word.isupper()': False, 'word.istitle()': True, 'word.isdigit()': False, 'postag': 'IN', 'postag[:2]': 'IN', 'BOS': True, '+1:word.lower()': 'returning', '+1:word.istitle()': False, '+1:word.isupper()': False, '+1:postag': 'VBG', '+1:postag[:2]': 'VB'}, {'bias': 1.0, 'word.lower()': 'returning', 'word[-3:]': 'ing', 'word[-2:]': 'ng', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'VBG', 'postag[:2]': 'VB', '-1:word.lower()': 'while', '-1:word.istitle()': True, '-1:word.isupper()': False, '-1:postag': 'IN', '-1:postag[:2]': 'IN', '+1:word.lower()': 'from', '+1:word.istitle()': False, '

In [0]:
# group B and I results

metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

In [0]:
# Model Optimization
# To improve quality and do hyperparameter tuning,
# optimizing the regularization parameters using randomized search and 3-fold cross-validation.

%%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05)
    
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter= 20,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

In [0]:
dump(rs, 'optimized_news_ner_model.joblib')
rs = load('optimized_news_ner_model.joblib')

In [0]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))
